## Calculation of efficiency of grid connected system
This calculation includes data of irradiance, efficiency of a PV module and efficiency of inverter given at hourly intervals.

In [1]:
import numpy as np
import pandas as pd
import math

Reading in the given dataset

In [2]:
df = pd.read_csv('sourcedata/week7.dat', parse_dates=True, index_col='DateT')
df.head()

,Gm,Eff,Eff_inv
DateT,,,
2005-01-01 01:00:00,0.0,0.0,0.0
2005-01-01 02:00:00,0.0,0.0,0.0
2005-01-01 03:00:00,0.0,0.0,0.0
2005-01-01 04:00:00,0.0,0.0,0.0
2005-01-01 05:00:00,0.0,0.0,0.0


In [3]:
df.size

26280

Given information about PV module and system

In [4]:
# PV module data
L = 1.56 # PV Module Length (meters)
W = 1.05 # PV Module Width (meters)
Pmpp_stc = 327 #[W]
Voc_stc = 67.6 # [V]
Isc_stc = 6.07 # [A]
Vmpp_stc = 57.3 # [V]
Impp_stc = 5.71 # [A]

# Standard data
Gstc = 1000 # [W/m2]
Tm_stc = 273+25 # [K]

# System information
SF = 1.1 # Sizing Factor [-]
Vmppy = 31 # Yearly Averaged Vmpp [V]  (used in 'Ns' calculations)
Eff_other = 0.98 # other efficiency losses [-]

# Load information
LOAD = 6350 # annual total energy consumption of the loads, connected to the pv system [kWh/yr] 

In [5]:
# calculation for one PV module

# module Area
A = L*W

# power generated by PV module [W]
df['Pdc'] = A*df['Gm'] * df['Eff']/100
max(df['Pdc'])

273.60173183864396

In [6]:
# for comparative purposes the maximum irradiance [W/m2]
max(df['Gm'])

888.40994558796604

In [7]:
# Total DC energy deliverd by one PV module  [kWh/yr]
Edc = sum(df['Pdc']) / 1000
Edc

302.99003772955865

In [8]:
# Calculation of number of modules needed to cover Load demand (based on Energy Balance)
# Number of modules is rounded up
Nt = math.ceil(LOAD*SF/Edc)
Nt

24

Inverter Sizing
    Pdc_stc = Nt * Pmpp_stc<br>
    Conditions
       * 1) Pdcmax_inv > Pdc_stc
       * 2) Pdc0 ~ Pdc_stc <br>
 

In [9]:
Pdc_stc = Nt * Pmpp_stc
Pdc_stc

7848

Thus selected inverter is SolarEdge SE7600A. From the sandia inverter database, 
 for this inveter the following parameters are obtained. 

In [10]:
Pdc0 = 7823.53 # [W]
Vdc0 = 402 # [V]

In [11]:
# number of PV modules connected in series
# this number is round down to nearest integer
Ns = math.floor(Vdc0/Vmppy)
Ns

12

In [12]:
# number of PV strings connected in parallel is then
Np = math.ceil(Nt/Ns)
Np

2

#### Performance Analysis of the grid connected PV system

In [13]:
# Instantenous AC power deliverd by the PV system [W]
df['Pac'] = Nt*Eff_other*df['Pdc']*df['Eff_inv']/100
max(df['Pac'])

6252.0120798213748

In [14]:
# Total AC Energy deliverd by the PV system in a year [kWh/yr] 
Eac_sys = sum(df['Pac'])/1000 
Eac_sys

6872.5079182590762

In [15]:
# Yearly system efficiency [%] 
Eff_sys = Eac_sys / sum(Nt*A*df['Gm']/1000) *100 
Eff_sys

18.36359476734836

In [16]:
# Yearly electricity yield [Wh/year per kWp]
EY = Eac_sys*1000/(Nt*Pmpp_stc)
EY

875.70182444687509

In [17]:
# checking if the system is well designed by the following condition:
Eac_sys > LOAD

True